In [24]:
# !pip install -r requirements.txt

In [1]:
import os
import base64
import requests
import json
from io import BytesIO
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


In [2]:

# Get OpenAI API Key from environment variable
api_key = os.getenv("OPENAI_API_KEY")
# os.environ["OPENAI_API_KEY"]
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

# Structured Output Schema
response_schema = ResponseSchema(
    name="object_availability",
    description="List of 'yes' or 'no' responses indicating whether the objects are present in the image.",
)

parser = StructuredOutputParser.from_response_schemas([response_schema])

metaprompt = """
 - Include a check for object availability: 'yes' or 'no'."""


# Function to encode the image
def encode_image_from_pil(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def prepare_inputs(message, image):
    base64_image = encode_image_from_pil(image)

    system_instruction = parser.get_format_instructions() + "\n\n" + metaprompt

    payload = {
        "model": "gpt-4o",
        "messages": [
            {"role": "system", "content": system_instruction},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": message},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            },
        ],
        "max_tokens": 800,
    }
    return payload


def request_gpt4v(message, image):

    payload = prepare_inputs(message, image)
    response = requests.post(
        "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
    )

    # Extract response content
    res = response.json()["choices"][0]["message"]["content"]
    print("res", res)
    # Parse response using structured output parser
    parsed_output = parser.parse(res)
    print(parsed_output)
    
    # return parsed_output

    # Convert to a list of integers
    availability = parsed_output.get("object_availability", [])
    if isinstance(availability, str):
        try:
            return availability  # Convert to integers
        except ValueError:
            raise ValueError("Unexpected non-numeric values in output.")
    else:
        raise ValueError("Output is not a valid list.")


# Pass an image (PIL format)qa_records when calling `request_gpt4v(message, image)`


# def get_prompt(obj_list, depth_caption):
#     prompt = (
#         "Each object in the image has a bright numeric ID at its center.\n\n"
#         "Identify the numeric ID for each of the following objects and return them in a Python list in the same order:\n\n"
#     )

#     for i in obj_list:
#         prompt += f"- {i}\n"

#     prompt += (
#         f"\nImage Caption about depth: \n{depth_caption}\n\n"  # Added depth caption to prompt
#         f"\nThere are exactly {len(obj_list)} objects to identify, so your answer must contain {len(obj_list)} numeric IDs.\n"
#         "If the object is present in the image, return the ID you see, even if uncertain.\n"
#         "Do not guess or assume an ID—only use IDs visible in the image.\n"
#         "try to answer confidently with what you have"
#         "If the object is completely missing from the image, return `-1`.\n"
#         "Return only a Python list of IDs, nothing else. Example: `[1, 2, -1, 4]`"
#     )

#     return prompt


def get_prompt(question_, depth_caption):
    prompt = (
        "Based on the image and its depth information, answer the following question with either 'yes' or 'no':\n\n"
        f"Question: {question_}\n\n"
        f"Image Caption about depth: \n{depth_caption}\n\n"
        "Do not guess or assume. Answer only based on the image content.\n"
        "Return only a Python list with a single 'yes' or 'no' string. Example: `['yes']`"
    )
    return prompt


def qa_records(
    ds, index, data_split,
    question="question", image="image",
    depth_caption="depth_caption", use_depth=False,
    pred_key="pred_res"
):
    record = ds[data_split][index]
    image_input = record[image]
    question_ = record[question]

    if use_depth:
        depth_caption_ = record[depth_caption]
        message = get_prompt(question_, depth_caption_)
        print(message)
    else:
        message = question_

    response = request_gpt4v(message, image_input)

    # Store the response using the specified key
    record[pred_key] =  response


    return response


In [3]:
def make_pope_prompt(record):
    """
    Build a prompt string for a single POPE dataset example.
    
    record: dict with keys
      - question (str)
      - depth_caption (str)         # if you still want to include depth info
    """
    # core metadata header
    prompt = (
        f"Question: {record['question']}\n"
    )

    # instruct the model on desired output
    prompt += (
        "Answer the above question with only 'yes' or 'no'.\n"
        "Do not add any other text or punctuation.\n"
    )

    # # optional: if you have a depth caption, append it
    # if 'depth_caption' in record and record['depth_caption']:
    #     prompt += f"\nDepth analysis: {record['depth_caption']}\n"

    return prompt


In [4]:
import os
import io
import pandas as pd
from PIL import Image as Image_PIL
import shutil
from datasets import load_dataset, Dataset, Image
from tqdm import tqdm
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file


True

In [5]:
# --- Configuration ---
hf_token = os.getenv("HF_TOKEN")
openai_api = os.getenv("OPENAI_API_KEY")
login(hf_token)

hf_user = "Rajarshi-Roy-research"
# dataset_name = "lmms-lab-POPE"
dataset_name = "lmms-lab-POPE-depth-captions"
commit_message = "Updating dataset with VLM"
img_cols = ["image"]
image_to_process = "image"
# data_split = "test"
data_split = "test_with_depth"

# --- Helper Functions ---
def save_image_from_bytes(image_data, idx, image_folder):
    """Saves an image from byte data or PIL Image and returns the saved file path."""
    if isinstance(image_data, list):
        image_data = image_data[0]
    print(idx)

    try:
        if isinstance(image_data, Image_PIL.Image):
            image = image_data
        elif isinstance(image_data, bytes):
            image = Image_PIL.open(io.BytesIO(image_data))
        else:
            print(f"Unsupported image data type at row {idx}: {type(image_data)}")
            return None

        image_path = os.path.join(image_folder, f"image_{idx}.jpg")
        image.save(image_path, format="JPEG")
        print(f"✅ Saved: {image_path}")
        return image_path
    except Exception as e:
        print(f"❌ Error saving image at row {idx}: {e}")
        return None


def push_to_hugging_face(ds, img_cols, hf_user, dataset_name, commit_message, hf_token, private=False):
    """Pushes the dataset to the Hugging Face Hub."""
    for img_col in img_cols:
        ds = ds.cast_column(img_col, Image())

    try:
        ds.push_to_hub(
            repo_id=f"{hf_user}/{dataset_name}",
            commit_message=commit_message,
            token=hf_token,
            private=private,
        )
        print(f"✅ Dataset successfully uploaded: {hf_user}/{dataset_name}")
    except Exception as e:
        print(f"❌ Error pushing to Hugging Face Hub: {e}")


# --- Main Script ---
# Clean up existing image folder
for img_col in img_cols:
    if os.path.exists(img_col):
        shutil.rmtree(img_col)
    os.makedirs(img_col)

# Load dataset
ds = load_dataset(f"{hf_user}/{dataset_name}")
print(ds[data_split][:1])

# Process images and add the image file path in the dataset
for img_col in img_cols:
    print(img_col)
    ds = ds.map(
        lambda example, idx: {
            img_col: save_image_from_bytes(example[img_col], idx, img_col),
            f"{img_col}_path": os.path.join(img_col, f"image_{idx}.jpg")  # Ensure correct column name for the path
        },
        with_indices=True,
        num_proc=1  # Adjust as needed for performance
    )

# Optionally push the dataset to Hugging Face Hub
# push_to_hugging_face(ds, img_cols, hf_user, dataset_name, commit_message, hf_token)

# Process and add captions (if needed)
try:
    pass
except FileNotFoundError:
    pass

# print(ds[data_split][:1])


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


{'id': ['2015'], 'question_id': ['2016'], 'question': ['Is there a chair in the image?'], 'answer': ['no'], 'image_source': ['COCO_val2014_000000501294'], 'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x1BF4CC44D00>], 'category': ['adversarial'], 'depth_caption': ['Closest: a black screen\n----\nMid Range: a TV\n----\nFarthest: a TV\n----\n']}
image


In [18]:
ds[data_split][1]["question"]

'Is there a chair in the image?'

In [ ]:
ds

DatasetDict({
    test_with_depth: Dataset({
        features: ['id', 'question_id', 'question', 'answer', 'image_source', 'image', 'category', 'depth_caption', 'image_path'],
        num_rows: 400
    })
})

In [31]:
data_split="test_with_depth"

In [32]:
ds[data_split][1]["question"]

'Is there a chair in the image?'

In [47]:
idx=1

question = ds[data_split][idx]["question"]
image = ds[data_split][idx]["image"]

request_gpt4v(question,image)

res ```json
{
	"object_availability": "yes"
}
```
{'object_availability': 'yes'}


'yes'

In [39]:
# encode_image_from_pil(image)
# encode_image_from_pil("image/image_1.jpg")

In [60]:
data_split

'test_with_depth'

In [64]:
qa_records(ds,1,data_split)
# qa_records(ds,1,data_split,use_depth=True)

res ```json
{
	"object_availability": "yes"
}
```
{'object_availability': 'yes'}


'yes'

In [66]:
from tqdm import tqdm

for idx, d in enumerate(tqdm(ds[data_split], desc="Running QA Records")):
    d["pred_res"] = qa_records(ds, idx, data_split)


Running QA Records:   0%|          | 0/400 [00:00<?, ?it/s]

Running QA Records:   0%|          | 1/400 [00:01<11:32,  1.74s/it]

res ```json
{
	"object_availability": "yes"
}
```
{'object_availability': 'yes'}


Running QA Records:   0%|          | 2/400 [00:03<12:08,  1.83s/it]

res ```json
{
	"object_availability": "yes"
}
```
{'object_availability': 'yes'}


...

res ```json
{
	"object_availability": "no"
}
```
{'object_availability': 'no'}


Running QA Records: 100%|█████████▉| 399/400 [11:26<00:01,  1.95s/it]

res ```json
{
	"object_availability": "yes"
}
```
{'object_availability': 'yes'}


Running QA Records: 100%|██████████| 400/400 [11:28<00:00,  1.72s/it]

res ```json
{
	"object_availability": "no"
}
```
{'object_availability': 'no'}


In [67]:
2+2

4

In [68]:
ds[data_split]

Dataset({
    features: ['id', 'question_id', 'question', 'answer', 'image_source', 'image', 'category', 'depth_caption', 'image_path'],
    num_rows: 400
})

In [ ]:
from tqdm import tqdm

for idx, d in enumerate(tqdm(ds[data_split], desc="Running QA Records")):
    d["pred_res_with_depth"] = qa_records(ds, idx, data_split,use_depth=True)
